### Pseudo code
1. get into directory and list all the folders in the directory 
2. get in each folder and list all the files 
3. open each file from row 5
4. merge into data frame 
5. merge all the files into a single dataframe and export into a new csv
6. perform logistic regression

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import os
import logging as lg
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
lg.basicConfig(filename="logs.txt",level=lg.INFO,format="%(asctime)s:%(levelname)s: %(message)s")

In [3]:

class mergeRawdata:
    def __init__(self, path):
        self.path = path

    def list_dir(self):
        try:
            files = os.listdir(self.path)
            lg.info("list_dir() called")
            lg.info(files)
        except Exception as e:
            lg.error(e)

    def merge_csv_opr(self):
        lg.info("merge_csv_opr() called")
        try:
            files = os.listdir(self.path)
            cwd_ = os.getcwd()
            name = "merged_folder"
            p = os.path.join(cwd_, name)
            os.makedirs(p, exist_ok=True)
            for file in files:
                if file.endswith('.pdf'):
                    lg.info(" .PDF file detected, skipping the file")
                    continue
                else:
                    list_of_csv_in_folder = []
                    folder = os.path.join(self.path, file)
                    csv_files = os.listdir(folder)
                    try:
                        for csv in csv_files:
                            csv_path = os.path.join(folder, csv)
                            try:
                                df = pd.read_csv(csv_path, skiprows=4)
                            except Exception as e:
                                lg.error(e)
                                lg.info("Anomalies deteced in csv file, hence skipping")
                                continue
                            list_of_csv_in_folder.append(df)
                        merged_df = pd.concat(list_of_csv_in_folder, ignore_index=True)
                        os.chdir(p)
                        name = f"{file}.csv"
                        merged_df.to_csv(name)
                    except Exception as e:
                        lg.error(e)

                        continue
        except Exception as e:
            lg.error(e)
        lg.info("Operation successful")
        print("All Merging Successful and is available in current dir in Merged_folder")







In [4]:
path = "C:\\Users\\Abhishek Mazumdar\\Downloads\\AReM"
obj = mergeRawdata(path)
obj.merge_csv_opr()
lg.info("Object created with path to folder")

All Merging Successful and is available in current dir in Merged_folder


<img src=Picture1.png width='300'>


Opening each folder and adding a column of categories same as file name and merging all of them into one final DataFrame

In [5]:
class df_operation:
    
    def add_category_merge(self):
        lg.info("add_category_merge function called")
        cwd = os.getcwd()
        csv_files = os.listdir(cwd)
        csv_list =[]
        try:
            for csv in csv_files:
                sub_path = os.path.join(cwd,csv)
                df = pd.read_csv(sub_path)
                df["category"] = csv.split(".")[0]
                lg.info(f"Category column : {csv} added to csv file {csv}")
                csv_list.append(df)
            final_dataframe = pd.concat(csv_list,ignore_index=True)
            os.chdir(cwd)
            final_dataframe.to_csv("final_csv.csv")
            lg.info("Final_csv file created by merging all the csv files")
        except Exception as e:
            lg.error(e)    

    

In [6]:
o = df_operation()
o.add_category_merge()

<img src=Picture2.png width="200">

final_csv Created 

Now Performing EDA operation on the final dataset followed my Logistic regression 


In [7]:
df = pd.read_csv("D:\\INEURON\\TASKS_Class\\merged_folder\\final_csv.csv")

In [8]:
df

,Unnamed: 0,Unnamed: 0.1,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,category
0,0,0,0,39.25,0.43,22.75,0.43,33.75,1.30,bending1
1,1,1,250,39.25,0.43,23.00,0.00,33.00,0.00,bending1
2,2,2,500,39.25,0.43,23.25,0.43,33.00,0.00,bending1
3,3,3,750,39.50,0.50,23.00,0.71,33.00,0.00,bending1
4,4,4,1000,39.50,0.50,24.00,0.00,33.00,0.00,bending1
...,...,...,...,...,...,...,...,...,...,...
41274,41274,7195,118750,31.50,1.66,12.50,3.20,14.25,4.44,walking
41275,41275,7196,119000,27.33,1.25,11.33,0.94,20.00,4.00,walking
41276,41276,7197,119250,37.80,7.68,14.20,2.48,17.25,0.83,walking
41277,41277,7198,119500,33.75,1.30,15.75,5.21,16.50,2.69,walking


In [9]:
df["category"].unique()

array(['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing',
       'walking'], dtype=object)

We need to perform Logistic regression where two types of classifications are basically performed 
1. Binary classification 
2. One VS Rest classification

I will be performing one VS rest classification to classify bending1 VS not bending1 (rest)

In [10]:
df = df.drop(["Unnamed: 0","Unnamed: 0.1","# Columns: time"],axis="columns")

In [11]:
df

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,category
0,39.25,0.43,22.75,0.43,33.75,1.30,bending1
1,39.25,0.43,23.00,0.00,33.00,0.00,bending1
2,39.25,0.43,23.25,0.43,33.00,0.00,bending1
3,39.50,0.50,23.00,0.71,33.00,0.00,bending1
4,39.50,0.50,24.00,0.00,33.00,0.00,bending1
...,...,...,...,...,...,...,...
41274,31.50,1.66,12.50,3.20,14.25,4.44,walking
41275,27.33,1.25,11.33,0.94,20.00,4.00,walking
41276,37.80,7.68,14.20,2.48,17.25,0.83,walking
41277,33.75,1.30,15.75,5.21,16.50,2.69,walking


In [15]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,f1_score

class Log_regression:
    def __init__(self,x_train,x_test,y_train,y_test):
        self.x_train= x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
    def best_solver(self):
        '''Gives the solver with best score or time '''
        class_type = len(np.unique(y_train))
        test_score = {}
        train_score = {}
        try:
            if class_type > 2:
                lg.info("Multiclass Classification requested")
                solvers = ['lbfgs', 'sag', 'saga', 'newton-cg'] #supported solver for Multiclass
                try:
                    for solver in solvers:
                        model = LogisticRegression(solver=solver)
                        model.fit(self.x_train,self.y_train)
                        training_score = model.score(self.x_train,self.y_train)
                        testing_score = model.score(self.x_test,self.y_test)
                        test_score[solver] = testing_score
                        train_score[solver] = training_score
                except Exception as e:
                    lg.error(e,"Error occured while testing solvers")


            elif class_type ==2 :
                lg.info("Binary Classification requested")
                solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'] #supported solver for binary class
                try:
                    for solver in solvers:
                        model = LogisticRegression(solver=solver)
                        model.fit(self.x_train,self.y_train)
                        training_score = model.score(self.x_train,self.y_train)
                        testing_score = model.score(self.x_test,self.y_test)
                        test_score[solver] = testing_score
                        train_score[solver] = training_score
                        confusion_matrix()
                        
                except Exception as e:
                    lg.error(e,"Error occured while testing solvers")
        except Exception as e:
            lg.error(e,"Problem in label data, no classes found")
        return test_score,train_score

        
        

In [16]:
class data_preprocessing:
    def __init__(self,dataframe):
        self.df = dataframe

    def preprocess(self,label_col = ''):
        '''Deletes NAN data, splits label and feature, label encoding, normalizing feature, train test split'''
        try:
            self.df.dropna(inplace=True)
            x , y = self.df.drop([label_col],axis='columns'), self.df[label_col]
            encode  = LabelEncoder()
            y_encoded = encode.fit_transform(y)
            scaler = StandardScaler()
            x_scaled = scaler.fit_transform(x)
            x_train,x_test,y_train,y_test = train_test_split(x_scaled,y_encoded,test_size=0.25,random_state=42)
            return x_train,x_test,y_train,y_test
        except Exception as e:
            lg.error(e,"Error occured while preprocessing")

        


In [17]:
objct = data_preprocessing(df)

In [18]:
x_train,x_test,y_train,y_test = objct.preprocess('category')

In [21]:
logi = Log_regression(x_train,x_test,y_train,y_test)

In [22]:
test_score,train_score = logi.best_solver()

In [23]:
test_score

{'lbfgs': 0.6715686274509803,
 'sag': 0.6715686274509803,
 'saga': 0.6715686274509803,
 'newton-cg': 0.6715686274509803}

In [24]:
train_score

{'lbfgs': 0.6712964475963267,
 'sag': 0.6713291284028889,
 'saga': 0.6713291284028889,
 'newton-cg': 0.6713291284028889}